In [1]:
import sys
sys.path.append('..')
from src.configs import DATACLASS_PATH, OUTPUT_PATH
import pickle
import json
from tqdm import tqdm

with open(DATACLASS_PATH / "without_last_cls.pkl", "rb") as f:
    recsys_data = pickle.load(f)

with open(OUTPUT_PATH / "bert4rec.default.without_last_cls.config.epoch=5-step=44928.json", "rb") as f:
    out = json.load(f)

In [48]:
# import pandas as pd
# customers = []
# target = []
# for c in tqdm(new_result):
#     customers.append(c)
#     target.append(",".join(new_result[c])) 
     
# result = pd.DataFrame({"customer": customers, "target": target})

100%|██████████| 115790/115790 [00:00<00:00, 1084076.00it/s]


In [49]:
# result

,customer,target
0,00002857-64cb-4112-afb9-aa8e6eb79eb4,"cdc3d70e-3f93-4c5d-8ea0-d152e9b6589b,da505f36-..."
1,0001ae29-bf87-411e-afed-3545f835ac52,"94dfbaec-94d2-466c-980d-8e32b87b05f8,21f1c4ad-..."
2,000265b1-95ad-487d-824f-30752b7486ff,"d2c3c7f2-4d8f-4d2f-a892-23fe812c9f7e,21f1c4ad-..."
3,0002c15b-d0a6-482d-9292-20ba2c2fcc61,"0bdd9d6e-4ef1-4b58-9b1a-34655612e3aa,ea00bae8-..."
4,00039ad1-1c9a-414a-b37e-0f34bc55c3c3,"3399e5a2-3aed-427d-97a9-5a8376a2ed46,ea00bae8-..."
...,...,...
115785,ffff742d-a685-4161-bc54-4b4dac33ec39,"69cc86ef-66dc-426b-86bb-41132f32af79,331b2246-..."
115786,ffff7ef9-9a24-4368-8ac5-354aacd765f3,"34a41a40-382c-4af5-b06b-54716aed8b4d,c03dfd05-..."
115787,ffffecab-6e82-4ada-a55e-a104f2f899f6,"9e6d9a61-b2d3-4c6e-9b82-3f650575a410,6c47bdfc-..."
115788,fffff859-d3e9-4e6f-840a-5112618a3b77,"aa2e5e7e-e9a7-4efd-9c28-a54b676e2514,6ab2a304-..."


In [50]:
# result.to_csv("result.csv", index=False)

In [8]:
# import pandas as pd
# recsys_data.dataframe['order_date'].astype("datetime64").groupby(pd.Grouper(freq='M'))

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [19]:
date = recsys_data.dataframe['order_date'].astype("datetime64")

recsys_data.dataframe['order_date'] = date.dt.to_period('M') 

In [27]:
bought_date_result = recsys_data.dataframe.groupby(['user_id', 'order_date']).agg({'item_id': 'count'}).reset_index().sort_values(['user_id', 'order_date']).drop_duplicates(['user_id'], keep='last')
bought_date_result.user_id = bought_date_result.user_id.apply(lambda x: recsys_data.cat2u[x] )

In [31]:
bought_date_result

,user_id,order_date,item_id
5,00002857-64cb-4112-afb9-aa8e6eb79eb4,2021-11,1
15,0001ae29-bf87-411e-afed-3545f835ac52,2021-11,11
24,000265b1-95ad-487d-824f-30752b7486ff,2021-11,1
25,0002c15b-d0a6-482d-9292-20ba2c2fcc61,2021-01,23
30,00039ad1-1c9a-414a-b37e-0f34bc55c3c3,2021-05,1
...,...,...,...
701008,ffff742d-a685-4161-bc54-4b4dac33ec39,2021-11,2
701010,ffff7ef9-9a24-4368-8ac5-354aacd765f3,2020-10,5
701014,ffffecab-6e82-4ada-a55e-a104f2f899f6,2020-10,4
701017,fffff859-d3e9-4e6f-840a-5112618a3b77,2021-07,1


In [36]:
bought_date_table= pd.Series(bought_date_result["item_id"].values,index=bought_date_result["user_id"]).to_dict()

In [42]:
new_result = {}
for i in tqdm(out):    
    bought_num = bought_date_table[i]
    bought_num = bought_num + 5
    if bought_num > 30 : 
        bought_num == 30
    new_result[i] = out[i][:bought_num]
    
    

100%|██████████| 115790/115790 [00:00<00:00, 239603.80it/s]


{'00002857-64cb-4112-afb9-aa8e6eb79eb4': ['cdc3d70e-3f93-4c5d-8ea0-d152e9b6589b',
  'da505f36-da1d-403c-904d-a54508c158ab',
  'ac000e4a-be9d-4a54-9d9c-6e3233bca1d2',
  'c03dfd05-3827-4fb5-bf0d-55e63665dc61',
  '27d11c9c-405e-4eff-8fea-c57ba235fea9',
  '40bb7525-2060-46f3-a9b7-fe7042c45a1b'],
 '0001ae29-bf87-411e-afed-3545f835ac52': ['94dfbaec-94d2-466c-980d-8e32b87b05f8',
  '21f1c4ad-843e-4305-a606-939f309c5686',
  '9d62f12c-fbaa-432d-86ad-67543457e69e',
  '7f7a7405-c43b-4cab-ac38-dc279aa12648',
  '33cf1554-6f5c-4949-a792-ca4820903192',
  '8be0fa07-9af1-43bd-b7f1-8db0e1daf03f',
  'aa2e5e7e-e9a7-4efd-9c28-a54b676e2514',
  'd2c3c7f2-4d8f-4d2f-a892-23fe812c9f7e',
  'd754f0c3-7962-4b07-824f-4ba2809a3da4',
  'c0faa132-d421-4e0a-97e0-deaf45ecb09c',
  '1edcc6c3-e45f-444f-960d-f674d4c66d92',
  '35d48314-f260-43ad-8cb7-99281602d8ea',
  '45bb3fcf-24b2-46a4-826f-0438284a0508',
  'b41e9195-2bfb-4358-b13d-31c5bcc49501',
  'e6dc7de5-b61f-43b0-87d5-a69cbf451a97',
  'bfa15ae0-0722-47da-b50c-f162083b49

In [15]:
result.to_csv("submission_bert.csv", index=False)

In [16]:
rec_result = []
for i in out:
    rec_result.extend(out[i])
    
print("total rec items", len(set(rec_result)))

rank = []
for check in set(rec_result):
    check = recsys_data.i2cat[check]    
    rank.append(recsys_data.dataframe["item_id"].value_counts().index.tolist().index(check))
    


total rec items 5449


In [4]:
sorted(set(rank))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [5]:
check = "b7278665-556c-4c0c-b8a3-825a9c56cb20"
check = recsys_data.i2cat[check]
recsys_data.dataframe["item_id"].value_counts().index.tolist().index(check)

58

In [18]:
len(recsys_data.dataframe.user_id.unique())

115790

In [47]:
for i in tqdm(range(1000)):
    recsys_data.dataframe["item_id"].value_counts().index.tolist().index(check)

100%|██████████| 1000/1000 [00:38<00:00, 26.15it/s]


In [6]:
recsys_data.dataframe.sort_values(by='order_date')

,id,order_date,item_id,sales_price,rating,department,store,city,district,user_id,sex,age_group,timestamp
3116642,ab676aa8-9982-4907-b148-9d27e5e0cd7e,2020-01-19 07:52:51,20638,165,15,22f2e7c7-6679-41fd-9c1d-6e9562b51bce,e04fce44-9700-422d-9c42-f7c649a98e52,eafc1307-bf22-4fd1-87d8-2528404afc12,947a4dd2-394f-498f-87cf-566a4f0c79dd,15134,NaN,NaN,1579420371
3967813,513f8331-c6ce-4506-8043-909855b80871,2020-04-29 00:00:39,11353,59,1,f57143f5-672f-4c78-89f1-242f7a2d8a65,ad4f2bbe-db20-4735-8d95-4015db4136ad,4b52005f-878a-4b2b-95ee-d9e2d719046a,d0c759fb-4f93-4538-93e3-d12c86cb004e,71090,NaN,5.0,1588118439
901820,513f8331-c6ce-4506-8043-909855b80871,2020-04-29 00:00:39,1706,59,1,f57143f5-672f-4c78-89f1-242f7a2d8a65,ad4f2bbe-db20-4735-8d95-4015db4136ad,4b52005f-878a-4b2b-95ee-d9e2d719046a,d0c759fb-4f93-4538-93e3-d12c86cb004e,71090,NaN,5.0,1588118439
2563158,a55dc624-1925-4e21-bf0e-0c32a3a31f06,2020-04-29 00:01:03,10294,0,1,c556530b-d0a9-4beb-bc19-7044962f9aa8,49171894-8056-4ffb-9242-472997c4b865,db83dcc1-85c9-44f9-a047-426b02e8e1a3,339fb39c-3a2b-45da-b036-8e2d7761c4b3,106781,NaN,4.0,1588118463
1038183,cd1309c8-82aa-4404-b4f7-4b08d138a9a1,2020-04-29 00:01:38,19898,379,1,fc537967-e742-4916-b487-8dedc17a11f5,3653eb5c-ff8d-444f-9067-589941992ac1,ef5ed033-a377-4ace-a66d-759bcc0b7557,1bd49d79-90c7-494f-a7b7-d03ada11c173,28611,NaN,NaN,1588118498
...,...,...,...,...,...,...,...,...,...,...,...,...,...
918165,86768d5d-6f7b-4389-94dd-eca16828e435,2021-12-31 22:59:09,18760,89,2,22f2e7c7-6679-41fd-9c1d-6e9562b51bce,f2d064b2-a928-48aa-be83-a5b3bfc02f41,db83dcc1-85c9-44f9-a047-426b02e8e1a3,b611c86e-0693-41bb-98d4-6de1b4a837f5,23298,NaN,5.0,1640991549
3503991,17954cf8-f487-4d66-afb3-6184c3386174,2021-12-31 23:00:55,22779,230,1,22f2e7c7-6679-41fd-9c1d-6e9562b51bce,177fd22d-d2e6-419f-b8d3-a8035c0bf5db,827e9e3a-30c8-40d0-8241-56773e6d8d9b,1332851c-3046-47e8-8039-70297bfbf240,33053,NaN,4.0,1640991655
916257,7a4caa73-53f5-465f-8b91-2473790d308e,2021-12-31 23:03:34,13347,469,1,f57143f5-672f-4c78-89f1-242f7a2d8a65,f2d064b2-a928-48aa-be83-a5b3bfc02f41,db83dcc1-85c9-44f9-a047-426b02e8e1a3,b611c86e-0693-41bb-98d4-6de1b4a837f5,103259,NaN,5.0,1640991814
846045,78561b25-df8e-46a8-a356-9dbf61935b3b,2021-12-31 23:34:26,29939,180,4,451ffc74-a780-4ae0-adba-ff8caa8f4b91,710ff6c2-00fa-4fae-9201-a4793754dbec,03de1a9a-d2cb-479b-8754-586c55ca33ca,5aa2d576-63b8-496c-b02c-ed304ac5862f,81915,A,5.0,1640993666


In [50]:
recsys_data.dataframe["item_id"].value_counts()[:10]

535013    604
249772    560
74827     535
550466    481
542518    431
564545    417
215838    410
432076    401
560021    400
309951    379
Name: item_id, dtype: int64

In [51]:
recsys_data.cat2i[535013]

'e07d82de-5218-4aa2-be64-348fab73da4a'

In [52]:
def sample_coverage(samples, items):
    not_in_count = 0
    for i in items:
        if i not in samples:
            not_in_count+=1
    
    not_coverage =  not_in_count / len(items)
    coverage = 1 - not_coverage
    return coverage, not_coverage, not_in_count

def popular_coverage(datacls, pop_ks = [10000]):
    ks, val_covs, test_covs = [], [], []
    
    for pop_k in pop_ks:
        pop = datacls.dataframe.item_id.value_counts()[:pop_k].index.tolist()
        val_all_items, test_all_items = [], []
        for i in tqdm(recsys_data.val_seqs):
            val_all_items.append(recsys_data.val_seqs[i][0])
            test_all_items.append(recsys_data.test_seqs[i][0])
        val_all_items = set(val_all_items)
        test_all_items = set(test_all_items)
        val_coverage = sample_coverage(pop, val_all_items)[0]
        test_coverage = sample_coverage(pop, test_all_items)[0]
        ks.append(pop_k)
        val_covs.append(val_coverage)
        test_covs.append(test_coverage)


    return ks, val_covs, test_covs


popular_coverage(recsys_data, pop_ks = list([30000]))

100%|██████████| 39208/39208 [00:00<00:00, 2756088.21it/s]


([30000], [0.19315258847529793], [0.19290742225786328])

In [53]:
from  src.datasets.negative_sampler import NegativeSampler

negative_sampler = NegativeSampler(
    train=recsys_data.train_seqs,
    val=recsys_data.val_seqs,
    test=recsys_data.test_seqs,
    user_count=recsys_data.num_users,
    item_count=recsys_data.num_items,
    sample_size=128
    if recsys_data.num_items * 0.8 > 3000
    else int(recsys_data.num_items * 0.8),  # Define the lower bound and upper bound
    dataclass_name=recsys_data.filename,
    seed=12345,
    method="popular",
)
negative_samples = negative_sampler.get_negative_samples()

Negatives samples exist. Loading.


In [54]:
negative_samples[0]

[535013,
 249772,
 74827,
 550466,
 542518,
 564545,
 215838,
 432076,
 560021,
 309951,
 21339,
 294036,
 392510,
 199299,
 320477,
 89294,
 305855,
 456956,
 131982,
 473052,
 511586,
 276015,
 434782,
 475227,
 32502,
 448819,
 363888,
 81829,
 356585,
 169825,
 304455,
 202414,
 562472,
 465512,
 187288,
 545000,
 91891,
 211514,
 24491,
 237083,
 294190,
 52913,
 437625,
 280364,
 6566,
 260595,
 67836,
 133719,
 513109,
 290213,
 117695,
 357752,
 346059,
 459436,
 156780,
 442357,
 361350,
 577858,
 192305,
 124550,
 134884,
 79018,
 437300,
 137339,
 12811,
 462875,
 100857,
 428616,
 278734,
 581930,
 241358,
 123582,
 164861,
 287946,
 216854,
 163691,
 442208,
 41608,
 143442,
 357348,
 136686,
 981,
 222,
 69776,
 449449,
 478980,
 155835,
 470139,
 69699,
 493558,
 380087,
 492243,
 272080,
 585656,
 415388,
 518534,
 344329,
 135897,
 21806,
 7127,
 583353,
 214492,
 245874,
 275565,
 503190,
 313858,
 230951,
 480825,
 215538,
 322936,
 316839,
 301617,
 64613,
 180720,
